In [4]:
BASEPATH = '/home/levaid/bigstorage/'
import os
import sys
sys.path.append(os.path.join(BASEPATH, 'open_lth'))
from open_lth import *
from pruning import sparse_global
from models import base, initializers
from lottery.desc import LotteryDesc
from foundations import hparams
import torch
import torch.nn.functional as F
import torch.nn as nn
import datetime
from copy import deepcopy
from torch.nn.utils import prune
import torchvision
import time
import glob
import pathlib

import numpy as np

from torchvision.datasets import CIFAR10

In [2]:
experiments = glob.glob('/home/levaid/bigstorage/open_lth_data/lottery_*')
for experiment_path in experiments:
    replicate_id = 'replicate_1'
    try:
        print(experiment_path)
        hparams = pathlib.Path(os.path.join(experiment_path, replicate_id, 'level_0/main/hparams.log')).read_text()
        print(len(glob.glob(f'{experiment_path}/*')))
        print(hparams)
    except:
        continue

SyntaxError: invalid syntax (<ipython-input-2-da03f242b3b1>, line 8)

In [5]:
snip_experiment = '/home/levaid/bigstorage/open_lth_data/lottery_1fdf6caccfa4b03ad942ad20d18f39e7'
sparse_experiment = '/home/levaid/bigstorage/open_lth_data/lottery_5b72a80f53ea30f58605786edb6517bd'

In [23]:
model_location =os.path.join(snip_experiment, 'replicate_1/level_7/main/model_ep5_it0.pth')

In [26]:
model = torch.load(model_location, map_location={'cuda:0': 'cuda:1'})
EPOCHS = 60
# EXPERIMENT_NAME = datetime.datetime.now().strftime(
#     "%Y_%m_%d_%H_%M_%S") + f'_{PRUNING_STRATEGY}_vgg{NETWORK_SIZE}'

In [27]:
train_dataset = CIFAR10(os.path.join(BASEPATH, 'open_lth_datasets/cifar10'), train=True, download=True,
                        transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
test_dataset = CIFAR10(os.path.join(BASEPATH, 'open_lth_datasets/cifar10'), train=False, download=True,
                       transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128)


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# os.mkdir(os.path.join(BASEPATH, 'open_lth_data', EXPERIMENT_NAME))

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.1, weight_decay=0.0001)

performance_metrics = []
for ep in range(EPOCHS):
    starttime = time.time()
    model.train()
    
    no_of_params = np.prod(model.state_dict()['layers.11.conv.weight'].cpu().numpy().shape)
    no_of_zeros = np.sum(model.state_dict()['layers.11.conv.weight'].cpu().numpy() == 0.)
    print(f'{no_of_zeros}/{no_of_params} = {1-no_of_zeros/no_of_params}')

    for it, (examples, labels) in enumerate(train_loader):
        examples = examples.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()

        loss = model.loss_criterion(model(examples), labels)

        for name, layer in model.named_parameters():
            if layer.requires_grad:
                layer.retain_grad()

        loss.backward()

        for name, layer in model.named_parameters():
            if layer.requires_grad:
                if name in model.grads:
                    model.grads[name] += np.abs(layer.grad.clone().cpu().numpy())
                else:
                    model.grads[name] = np.abs(layer.grad.clone().cpu().numpy())
                # print(layer.grad)

        optimizer.step()

    correct = 0
    model.eval()
    for it, (examples, labels) in enumerate(test_loader):
        examples = examples.to(device=device)
        labels = labels.to(device=device)

        correct += torch.sum(torch.argmax(model(examples),
                                          dim=1) == labels).cpu().numpy()
    performance_metrics += [(f'test_accuracy,{ep},{correct/10000.0}')]
    print(performance_metrics[-1] + f' time: {time.time()-starttime:.2f}s')
    
    if ep == 40:
        print('reducing LR')
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 10

    # with open(os.path.join(BASEPATH, f'open_lth_data/{EXPERIMENT_NAME}/level_{level}_perf.log'), 'w') as f:
    #     for line in performance_metrics:
    #         f.write(line + '\n')

# torch.save(model, (os.path.join(BASEPATH, f'open_lth_data/{EXPERIMENT_NAME}/model_resnet_{NETWORK_SIZE}_level_{level}_.pth')))


Files already downloaded and verified
Files already downloaded and verified
2358433/2359296 = 0.00036578708224821455
test_accuracy,0,0.6444 time: 23.22s
318580/2359296 = 0.8649681939019097
test_accuracy,1,0.7146 time: 22.97s
318580/2359296 = 0.8649681939019097
test_accuracy,2,0.7647 time: 23.41s
318580/2359296 = 0.8649681939019097
test_accuracy,3,0.7046 time: 23.39s
318580/2359296 = 0.8649681939019097
test_accuracy,4,0.6872 time: 23.15s
318580/2359296 = 0.8649681939019097
test_accuracy,5,0.7379 time: 23.27s
318580/2359296 = 0.8649681939019097
test_accuracy,6,0.7603 time: 22.56s
318580/2359296 = 0.8649681939019097
test_accuracy,7,0.7171 time: 22.30s
318580/2359296 = 0.8649681939019097
test_accuracy,8,0.7784 time: 22.37s
318580/2359296 = 0.8649681939019097
test_accuracy,9,0.7564 time: 22.96s
318580/2359296 = 0.8649681939019097
test_accuracy,10,0.7958 time: 23.68s
318580/2359296 = 0.8649681939019097
test_accuracy,11,0.7746 time: 23.39s
318580/2359296 = 0.8649681939019097
test_accuracy,12,

KeyboardInterrupt: 

123140